In [1]:
import sys
from pathlib import Path

p = str(Path.home() / "miniconda3/envs/bias-pmi/lib/python3.9/site-packages")
sys.path.append(p)

In [2]:
import pandas as pd
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt

from scripts.utils.figures import pearson_cor, scatter_color_plt, scatter_plt

In [3]:
pd.set_option("display.max_columns", 100)

## Data

Read final bias results

In [4]:
df_final = pd.read_csv("results/figures_data.csv")

In [5]:
df_final[["experiment","corpus","bias"]].value_counts().sort_index()

experiment          corpus    bias  
glasgow-gender      subs      gender    4668
                    wiki2014  gender    4642
names-gender        subs      gender      50
                    wiki2014  gender      50
occupations-gender  subs      gender      44
                    wiki2014  gender      44
dtype: int64

## Correlations

In [6]:
bias_and_se = [
    ("wefat_score_sgns", None),
    ("wefat_score_sgns", "se_sgns"),
    ("wefat_score_glovewc", None),
    ("wefat_score_glovewc", "se_glovewc"),
    ("dpmi", None),
    ("dpmi", "lor_se"),
]

In [7]:
corpus_dict = {"subs" : "OpenSubtitles", "wiki2014" : "Wikipedia"}
metrics_dict = {"dpmi" : "PMI", "wefat_score_glovewc" : "GloVe", 
                "wefat_score_sgns": "SGNS"}

In [8]:
res = []

for metric, se in bias_and_se:
    res_ = df_final.groupby(["corpus","experiment"], as_index=False).apply(
        lambda d: pearson_cor(d, "score", metric, weight_se_var=se))
    res_["correlation"] = "Weighted $r$" if se else "$r$"
    res_["metric"] = metric
    res_ = res_.replace({"corpus": corpus_dict, "metric": metrics_dict})
    res_["experiment"] = res_["experiment"].str.title()
    res_.columns = res_.columns.str.title()
    res += [res_.copy()]

df_res = pd.concat(res)
df_res.rename(columns={None: "value"}, inplace=True)
df_res = df_res.set_index(["Corpus", "Experiment", "Correlation", "Metric"],).unstack()
df_res.columns = df_res.columns.droplevel(0)
df_res = df_res[["PMI","GloVe","SGNS"]]

In [9]:
df_res.round(3)

Metric                                           PMI  GloVe   SGNS
Corpus        Experiment         Correlation                      
OpenSubtitles Glasgow-Gender     $r$           0.578  0.489  0.550
                                 Weighted $r$  0.577  0.686  0.723
              Names-Gender       $r$           0.796  0.738  0.810
                                 Weighted $r$  0.840  0.819  0.769
              Occupations-Gender $r$           0.655  0.672  0.787
                                 Weighted $r$  0.813  0.832  0.886
Wikipedia     Glasgow-Gender     $r$           0.499  0.442  0.504
                                 Weighted $r$  0.440  0.590  0.663
              Names-Gender       $r$           0.783  0.744  0.765
                                 Weighted $r$  0.755  0.786  0.762
              Occupations-Gender $r$           0.687  0.703  0.700
                                 Weighted $r$  0.788  0.672  0.783

In [10]:
df_ = df_res.query(
    "Experiment in ['Glasgow-Gender','Occupations-Gender','Names-Gender']")
print(
    df_.style.format(precision=2).to_latex(
        clines="skip-last;data", multicol_align="|c|", hrules=True
        # column_format="c", multirow_align="naive", multicol_align="c"
))

\begin{tabular}{lllrrr}
\toprule
 &  & Metric & PMI & GloVe & SGNS \\
Corpus & Experiment & Correlation &  &  &  \\
\midrule
\multirow[c]{6}{*}{OpenSubtitles} & \multirow[c]{2}{*}{Glasgow-Gender} & $r$ & 0.58 & 0.49 & 0.55 \\
 &  & Weighted $r$ & 0.58 & 0.69 & 0.72 \\
\cline{2-6}
 & \multirow[c]{2}{*}{Names-Gender} & $r$ & 0.80 & 0.74 & 0.81 \\
 &  & Weighted $r$ & 0.84 & 0.82 & 0.77 \\
\cline{2-6}
 & \multirow[c]{2}{*}{Occupations-Gender} & $r$ & 0.66 & 0.67 & 0.79 \\
 &  & Weighted $r$ & 0.81 & 0.83 & 0.89 \\
\cline{1-6} \cline{2-6}
\multirow[c]{6}{*}{Wikipedia} & \multirow[c]{2}{*}{Glasgow-Gender} & $r$ & 0.50 & 0.44 & 0.50 \\
 &  & Weighted $r$ & 0.44 & 0.59 & 0.66 \\
\cline{2-6}
 & \multirow[c]{2}{*}{Names-Gender} & $r$ & 0.78 & 0.74 & 0.77 \\
 &  & Weighted $r$ & 0.75 & 0.79 & 0.76 \\
\cline{2-6}
 & \multirow[c]{2}{*}{Occupations-Gender} & $r$ & 0.69 & 0.70 & 0.70 \\
 &  & Weighted $r$ & 0.79 & 0.67 & 0.78 \\
\cline{1-6} \cline{2-6}
\bottomrule
\end{tabular}



## Plots

In [11]:
metrics_info = {
    "wefat_score_sgns": {
        "label": "SGNS",
        "pvalue_col": "sgns_pvaluecor",
        "se_and_ci_cols": [[None, None], ["se_sgns", ["lower_sgns","upper_sgns"]]],
        # "se_and_ci_cols": [[None, None], ],
        },
    "wefat_score_glovewc": {
        "label": "GloVe",
        "pvalue_col": "glovewc_pvaluecor",
        "se_and_ci_cols": [[None, None], ["se_glovewc", ["lower_glovewc","upper_glovewc"]]],
        # "se_and_ci_cols": [[None, None], ],
        },
    "dpmi": {
        "label": "PMI",
        "pvalue_col": "lor_pvaluecor",
        "se_and_ci_cols": [[None, None], ["lor_se", ["dpmi_lower","dpmi_upper"]]],
        },
    }

In [12]:
corpora_info = {
    "wiki2014": {"label": "Wikipedia"},
    "subs": {"label": "OpenSubtitles"},
}

In [13]:
experiments_info = {
    "occupations-gender": {
        "col_label": r"% of workers in occupation who are women",
        "bias_label": "Female bias",
        "point_size": 35,
        "edgecolor": "black",
        "error_bars": True,
    },
    "names-gender": {
        "col_label": r"% of people with name who are women",
        "bias_label": "Female bias",
        "point_size": 35,
        "edgecolor": "black",
        "error_bars": True,
    },
    "glasgow-gender": {
        "bias_label": "Female bias",
        "col_label": r"Human judgement of gender association",
        "point_size": 2,
        "edgecolor": None,
        "error_bars": False,
    },
}

In [14]:
df_settings = df_final[["corpus", "experiment"]].drop_duplicates()

In [15]:
plot_files = Path("results/plots").glob("*.png")
plot_files = sorted(list(plot_files))

In [16]:
grid_info = {
    "experiments": ["names-gender", "occupations-gender", "glasgow-gender"],
    "metrics": ["PMI","GloVe","SGNS"],
    "metric_types": ["uw", "w"],
    "corpora": ["wiki2014"], #["subs","wiki2014"],
}

In [17]:
def concat_tile(lista_2d: list):
    return cv.vconcat([cv.hconcat(list_h) for list_h in lista_2d])

### Experiments

In [18]:
# save PNGs
dpi = 300
width = 1500
height = 1150
plt.ioff()

for _, (corpus, experiment) in df_settings.iterrows():
    for metric, metric_info in metrics_info.items():
        for se_col, ci_cols in metric_info["se_and_ci_cols"]:
            
            experiment_info = experiments_info[experiment]
            corpus_info = corpora_info[corpus]
            metric_type = "w" if se_col else "uw"
            xlabel = experiment_info['col_label']
            ylabel = f"{experiment_info['bias_label']} ({metric_info['label']})"
            corpus_name = corpus_info["label"]
            error_bars = experiment_info["error_bars"]
            point_size = experiment_info["point_size"]
            edgecolor = experiment_info["edgecolor"]
            ci_cols = ci_cols if error_bars else None
            outfile = f"results/plots/{corpus}_{experiment}_{metric_info['label']}_{metric_type}.png"

            df_ = df_final.query("experiment == @experiment & corpus == @corpus")

            fig_, ax_ = scatter_color_plt(
                "score", metric, df_, weight_se_var=se_col, error_cols=ci_cols,
                xlabel=xlabel, ylabel=ylabel, title=None, point_size=point_size,
                edgecolor=edgecolor, print_pearson=False
            )
            fig_.set_size_inches(width/dpi, height/dpi)
            plt.savefig(outfile, dpi=dpi, bbox_inches='tight')
            plt.close(fig_)


In [19]:
plot_files_wefat = [f for f in plot_files if not f.stem.startswith("pvalues")]
plot_files_dict = {}
for f in plot_files_wefat:
    f_name = f.stem
    plot_files_dict[f_name] = str(f)

In [20]:
img_lists = {}
img_grids = {}
for experiment in grid_info["experiments"]:
    img_lists[experiment] = []
    for corpus in grid_info["corpora"]:
        for metric_type in grid_info["metric_types"]:
            img_row = []
            for metric in grid_info["metrics"]:
                k = f"{corpus}_{experiment}_{metric}_{metric_type}"
                f = plot_files_dict[k]
                img_ = cv.imread(f)
                img_ = cv.resize(img_, (1401, 1068))
                img_row.append(img_)
            img_lists[experiment].append(img_row)
    img_grids[experiment] = concat_tile(img_lists[experiment])
    cv.imwrite(f"results/plots/grid_{experiment}.png", img_grids[experiment])


### Permutation p-values

In [21]:
# save PNGs
dpi = 300
width = 2100
height = 1050
plt.ioff()

for _, (corpus, experiment) in df_settings.iterrows():
    for metric, metric_info in metrics_info.items():

        experiment_info = experiments_info[experiment]
        corpus_info = corpora_info[corpus]
        corpus_name = corpus_info["label"]
        point_size = experiment_info["point_size"]
        edgecolor = experiment_info["edgecolor"]
        pvalue_col = metric_info["pvalue_col"]
        xlabel = f"{experiment_info['bias_label']} ({metric_info['label']})"
        ylabel = f"{experiment_info['bias_label']} p-value ({metric_info['label']})"
        outfile = f"results/plots/pvalues_{corpus}_{experiment}_{metric_info['label']}.png"
        
        df_ = df_final.query("experiment == @experiment & corpus == @corpus")

        fig_, ax_ = scatter_plt(
            metric, pvalue_col, df_, xlabel=xlabel, ylabel=ylabel, 
            title=None, point_size=point_size, edgecolor=edgecolor)
        fig_.set_size_inches(width/dpi, height/dpi)
        plt.savefig(outfile, dpi=dpi, bbox_inches='tight')
        plt.close(fig_)

In [22]:
plot_files_pvalues = [f for f in plot_files if f.stem.startswith("pvalues")]
plot_files_dict = {}
for f in plot_files_pvalues:
    f_name = f.stem
    plot_files_dict[f_name] = str(f)

In [23]:
img_list = []
for experiment in grid_info["experiments"]:
    for corpus in grid_info["corpora"]:
        img_row = []
        for metric in grid_info["metrics"]:
            k = f"pvalues_{corpus}_{experiment}_{metric}"
            f = plot_files_dict[k]
            img_ = cv.imread(f)
            img_ = cv.resize(img_, (width, height))
            img_row.append(img_)
        img_list.append(img_row)

img_grid = concat_tile(img_list)

In [24]:
cv.imwrite(f"results/plots/grid_pvalues.png", img_grid)

True

In [25]:
# example pvalues figure
file_pmi = plot_files_dict["pvalues_wiki2014_names-gender_PMI"]
file_we = plot_files_dict["pvalues_wiki2014_names-gender_SGNS"]
img_pmi = cv.imread(file_pmi)
img_we = cv.imread(file_we)
img_pmi = cv.resize(img_pmi, (width, height))
img_we =  cv.resize(img_we, (width, height))
img_grid = cv.vconcat([img_we, img_pmi])

cv.imwrite(f"results/plots/grid_pvalues_wiki2014_names-gender.png", img_grid)

True

In [26]:
pvalues_sgns = df_final.query(
    "experiment == 'names-gender' & corpus == 'wiki2014'")["sgns_pvaluecor"]
sum(pvalues_sgns < .1)

1

## Save results summary

In [27]:
cols_to_save = [
    "word", "corpus", "experiment", "bias", "score",
    "dpmi", "lor_se", "dpmi_lower", "dpmi_upper", "lor_pvalue", "lor_pvaluecor",
    "wefat_score_sgns", "se_sgns", "lower_sgns", "upper_sgns", "sgns_pvalue", "sgns_pvaluecor",
    "wefat_score_glovewc", "se_glovewc", "lower_glovewc", "upper_glovewc", "glovewc_pvalue", "glovewc_pvaluecor",
]
df_to_save = (
    df_final[cols_to_save]
    .rename(
        columns={"score": "external_score", "dpmi": "bias_pmi",
                 "wefat_score_sgns": "bias_sgns", "wefat_score_glovewc": "bias_glovewc"})
    .sort_values(["corpus", "experiment", "word", ])
    .reset_index(drop=True)
)
df_to_save.to_csv("results/experiments_results.csv", index=False)